# Study force-controlled single facet

TODO
 - Section 1 - costruct a displacement controlled model and show the load displacement curve for
   sliding
 - Section 2 - with the identified 

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets import FETS3D8H
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_utils.api import Model

In [5]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

In [6]:
# plt.rcParams['text.usetex']=True
# #plt.rc('text.latex', preamble=r'\usepackage[bitstream-charter]{mathdesign}')
# plt.rcParams['font.size'] = 7

# # Times, Palatino, New Century Schoolbook, Bookman, Computer Modern Roman
# plt.rc('font',**{'family':'serif','serif':['Computer Modern Roman']}) 

# import matplotlib
# matplotlib.use('pgf')
# plt.rcParams["pgf.texsystem"] = "xelatex"
# plt.rcParams["pgf.rcfonts"] = False

# preamble = r'''\usepackage[utf8]{inputenc} %unicode support
# \usepackage[czech]{babel}
# \usepackage[T1]{fontenc}
# \DeclareMathAlphabet{\pazocal}{OMS}{zplm}{m}{n}
# \usepackage{calrsfs}
# \usepackage{amsmath}
# \usepackage{bm}
# \usepackage[bitstream-charter]{mathdesign}
# %\usepackage{xcolor}
# '''
# plt.rc('text.latex', preamble=preamble)
# plt.rcParams["pgf.preamble"] = preamble

In [8]:
compression_range = np.array(np.linspace(0,-30,3))

m_list = []

for compression in compression_range: # compression_range:

    n_e = 1
    L_x = 1.0
    
    xd_lower = XDomainFEGrid(coord_min=(0, -1),
                              coord_max=(1, 0),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    xd_upper = XDomainFEGrid(coord_min=(0, 0),
                              coord_max=(1, 1),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    
    m1 = MATS2DElastic(E=40000, nu=0.2)
    m2 = MATS2DElastic(E=40000, nu=0.2)

    xd12 = XDomainFEInterface(
        I=xd_lower.mesh.I[-1, :],
        J=xd_upper.mesh.I[0, :],
        fets=FETS1D52ULRH()
    )
    
    material_params = dict(
         E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
         E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)
    
#    material_params = dict(
#    E_T=10000, gamma_T=4000, K_T=0, S_T=0.0001, c_T=2, bartau=5, 
#    E_N=20000, S_N=10000000.5, c_N = 1, m = 0.008, f_t=5, f_c=150, f_c0=100, eta=0.0
#    )
    
    bond_m = Slide34(**material_params)
    
    m = TStepBC(
        domains=[
            (xd_lower, m1),
            (xd_upper, m2),
            (xd12, bond_m),
        ],
    )
    
    tf_force = TFBilinear(loading_ratio=0.99, time_ratio=0.5)
    tf_disp = TFBilinear(loading_ratio=0.00, time_ratio=0.5)
    
    load_dim = 0 # 
    fixed_dim = 1 # 0 - x, 1 - y
    
    upper_control_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
    upper_control_dofs = upper_control_slice.dofs
    upper_control = [BCDof(var='u', dof=dof, value=0.4 , time_function=tf_disp) 
                     for dof in upper_control_dofs ]
    
    F = compression / len(upper_control_dofs)

    upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
             var='u', dims=[fixed_dim], value=0)
    upper_compression_dofs = upper_compression_slice.dofs
    upper_compression = [BCDof(var='f', dof=dof, value = F , time_function=tf_force ) 
                     for dof in upper_compression_dofs ]
    lower_fixed = BCSlice(slice=xd_lower.mesh[:, :, :, :],
             var='u', dims=[0,1], value=0)
    bc1 = upper_control + upper_compression + [lower_fixed]

    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = False # set false to avoid the prints during each increment
    s.tloop.k_max = 50
    s.tline.step = 0.01
    s.reset()
    s.run()
    m_list.append(m)

In [9]:
import matplotlib.pylab as plt
fig, ((ax1,  ax2, ax3,  ax4)) =  plt.subplots(1,4, figsize=(16/2.54, 6/2.54)) #, constrained_layout=False)
fig.canvas.header_visible=False
fig.tight_layout()

max_F = []

for i in range(len(m_list)):
    
    m = m_list[i]
    F_to = m.hist.F_t
    U_to = m.hist.U_t
    F_l_t = np.sum(F_to[:, upper_control_dofs], axis=-1)
    U_r_t = np.average(U_to[:, upper_control_dofs], axis=-1)
    
    states_t = [states_t[2] for states_t in m.hist.state_vars]
    var_names = states_t[0].keys()
    EpsSig_t = {
        var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
        for var_name in var_names 
    }
    
    time = np.arange(len(F_l_t))
    
    ax1.plot(time/time[-1], EpsSig_t['tau_pi_x'][:,0,0], label=str(compression_range[i] ))
    ax1.set_xlabel('time')
    ax1.set_ylabel('$\sigma_T$')
    ax1.legend()

    ax2.plot(time/time[-1], -EpsSig_t['sig_pi'][:,0,0])
    ax2.set_xlabel('time')
    ax2.set_ylabel('$\sigma_N$')
    
    ax3.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
    ax3.set_xlabel('s_x')
    ax3.set_ylabel('$\omega_T$')
    
    
    ax4.plot(U_r_t, -EpsSig_t['omega_N'][:,0,0])
    ax4.set_xlabel('s_x')
    ax4.set_ylabel('$\omega_N$')
    
    max_F.append(max(abs( -EpsSig_t['tau_pi_x'][:,0,0])))

#fname= 'ElsevierFont.pdf'
#plt.savefig(fname, bbox_inches='tight', pad_inches=0.01/2.54, dpi=300)
##get the link to the PDF
#from IPython.display import FileLink
#FileLink(fname)
#p#lt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
number_of_cycles=1
shift_cycles=1

tf_cyclic = TFCyclicNonsymmetricConstant(number_of_cycles=number_of_cycles, unloading_ratio=0.05, shift_cycles=shift_cycles)
tf_confinement = TFBilinear(loading_ratio=0.99, time_ratio=1/(number_of_cycles+shift_cycles))
tf_cyclic.interact()

In [16]:
m = m_list[1]
if False:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'strain'
    strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [17]:
max_F

[10.174369740579188, 10.775699740336076, 6.108358949452149]

In [26]:
compression_range = np.array(np.linspace(0,-80,3))

m_list_cyclic = []
max_F = [10.174369740579188, 10.775699740336076, 6.108358949452149]

for i in range(len(max_F)):

    n_e = 1
    L_x = 1.0
    
    xd_lower = XDomainFEGrid(coord_min=(0, -1),
                              coord_max=(1, 0),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    xd_upper = XDomainFEGrid(coord_min=(0, 0),
                              coord_max=(1, 1),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    
    m1 = MATS2DElastic(E=40000, nu=0.2)
    m2 = MATS2DElastic(E=40000, nu=0.2)

    xd12 = XDomainFEInterface(
        I=xd_lower.mesh.I[-1, :],
        J=xd_upper.mesh.I[0, :],
        fets=FETS1D52ULRH()
    )
    
    material_params = dict(
    E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
    E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)
    
    bond_m = Slide34(**material_params)
    
    m = TStepBC(
        domains=[
            (xd_lower, m1),
            (xd_upper, m2),
            (xd12, bond_m),
        ],
    )
    number_of_cycles=1
    shift_cycles=1
    
    tf_cyclic = TFCyclicNonsymmetricConstant(number_of_cycles=number_of_cycles, unloading_ratio=0.05, shift_cycles=shift_cycles)
    #tf_cyclic = TFBilinear(loading_ratio=0.01, time_ratio=0.5)
    tf_confinement = TFBilinear(loading_ratio=1, time_ratio=1/(number_of_cycles+shift_cycles))
    
    load_dim = 0 # 
    fixed_dim = 1 # 0 - x, 1 - y
    
    
    upper_control_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
    upper_control_dofs = upper_control_slice.dofs
    
    F_tau = -0.7 * max_F[i] / len(upper_control_dofs)
    
    upper_control = [BCDof(var='f', dof=dof, value=F_tau , time_function=tf_cyclic) 
                     for dof in upper_control_dofs ]
    
    F = compression_range[i] / len(upper_control_dofs)

    upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
             var='u', dims=[fixed_dim], value=0)
    upper_compression_dofs = upper_compression_slice.dofs
    upper_compression = [BCDof(var='f', dof=dof, value = F , time_function=tf_confinement) 
                     for dof in upper_compression_dofs ]
    
    lower_fixed = BCSlice(slice=xd_lower.mesh[:, :, :, :],
             var='u', dims=[0,1], value=0)
    
    bc1 = upper_control + upper_compression + [lower_fixed]

    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = True # set false to avoid the prints during each increment
    s.tloop.k_max = 50
    s.tline.step = 0.01
    s.reset()
    s.run()
    m_list_cyclic.append(m)

t:	 0.00(0), 
	 0.01(0), 
	 0.02(0), 
	 0.03(0), 
	 0.04(0), 
	 0.05(0), 
	 0.06(0), 
	 0.07(0), 
	 0.08(0), 
	 0.09(0), 
	 0.10(0), 
	 0.11(0), 
	 0.12(0), 
	 0.13(0), 
	 0.14(0), 
	 0.15(0), 
	 0.16(0), 
	 0.17(0), 
	 0.18(0), 
	 0.19(0), 
	 0.20(0), 
	 0.21(0), 
	 0.22(0), 
	 0.23(0), 
	 0.24(0), 
	 0.25(0), 
	 0.26(0), 
	 0.27(0), 
	 0.28(0), 
	 0.29(0), 
	 0.30(0), 
	 0.31(0), 
	 0.32(0), 
	 0.33(0), 
	 0.34(0), 
	 0.35(0), 
	 0.36(0), 
	 0.37(0), 
	 0.38(0), 
	 0.39(0), 
	 0.40(0), 
	 0.41(0), 
	 0.42(0), 
	 0.43(0), 
	 0.44(0), 
	 0.45(0), 
	 0.46(0), 
	 0.47(0), 
	 0.48(0), 
	 0.49(0), 
	 0.50(0), 
	 0.51(1), 
	 0.52(1), 
	 0.53(1), 
	 0.54(1), 
	 0.55(1), 
	 0.56(1), 
	 0.57(1), 
	 0.58(1), 
	 0.59(1), 
	 0.60(1), 
	 0.61(7), 
	 0.62(8), 
	 0.63(8), 
	 0.64(8), 
	 0.65(8), 
	 0.66(8), 
	 0.67(9), 
	 0.68(9), 
	 0.69(9), 
	 0.70(9), 
	 0.71(10), 
	 0.72(10), 
	 0.73(10), 
	 0.74(11), 
	 0.75(12), 
	 0.76(1), 
	 0.77(1), 
	 0.78(1), 
	 0.79(1), 
	 0.80(1), 
	 0.81(1), 
	 0.82(1)

RuntimeError: Factor is exactly singular

In [28]:
import matplotlib.pylab as plt
fig, ((ax1,  ax2, ax3,  ax4)) =  plt.subplots(1,4, figsize=(16/2.54, 6/2.54)) #, constrained_layout=False)
fig.canvas.header_visible=False
fig.tight_layout()

for i in range(len(m_list_cyclic)):
    m = m_list_cyclic[i]
    F_to = m.hist.F_t
    U_to = m.hist.U_t
    upper_control_dofs = [bc.dof for bc in upper_control]
    F_l_t = np.sum(F_to[:, upper_control_dofs], axis=-1)
    U_r_t = np.average(U_to[:, upper_control_dofs], axis=-1)
    
    states_t = [states_t[2] for states_t in m.hist.state_vars]
    var_names = states_t[0].keys()
    EpsSig_t = {
        var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
        for var_name in var_names 
    }
    time = np.arange(len(F_l_t))
    
    ax1.plot(U_r_t, EpsSig_t['tau_pi_x'][:,0,0], label=str(compression_range[i]))
    ax1.set_xlabel('s_x')
    ax1.set_ylabel('$\sigma_T$')
    ax1.legend()
    
    ax2.plot(time, EpsSig_t['sig_pi'][:,0,0])
    ax2.set_xlabel('s_x')
    ax2.set_ylabel('$\sigma_N$')
    
    ax3.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
    ax3.set_xlabel('s_x')
    ax3.set_ylabel('$\omega_T$')
    
    
    ax4.plot(U_r_t, -EpsSig_t['omega_N'][:,0,0])
    ax4.set_xlabel('s_x')
    ax4.set_ylabel('$\omega_N$')
    
    #max_F.append(max(abs( -EpsSig_t['tau_pi_x'][:,0,0])))
    
#fname= 'ElsevierFont.pdf'
#plt.savefig(fname, bbox_inches='tight', pad_inches=0.01/2.54, dpi=300)
#
## get the #link to the PDF
#from IPython.display import FileLink
#FileLink(fname)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
tf_cyclic.interact()

In [13]:
m1 = m_list_cyclic[0]

In [14]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'strain'
    strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [39]:
var_names

dict_keys(['w_pi', 's_pi_x', 's_pi_y', 'z', 'alpha_x', 'alpha_y', 'omega_T', 'omega_N', 'sig_pi', 'tau_pi_x', 'tau_pi_y', 'Z', 'X_x', 'X_y', 'Y_T', 'Y_N'])

In [16]:
tf_disp.interact()